In [76]:
import wikipedia
import os
import re
import random
import nltk
from nltk.tokenize import sent_tokenize
from joblib import Parallel, delayed
from tqdm import tqdm
import sys

nltk.download("punkt")
wikipedia.set_lang("pt")

PASTA_DESTINO = r'C:\Users\Administrator\Desktop\Repositórios\SimCSE\textos_wikipedia'
os.makedirs(PASTA_DESTINO, exist_ok=True)

# Garante que o nome do arquivo seja válido
def limpar_nome_arquivo(nome):
    return re.sub(r"[^\w\s-]", "", nome).strip().replace(" ", "_") + ".txt"

def contar_total_linhas_txt(diretorio):
    total_linhas = 0
    total_arquivos = 0

    for root, dirs, files in os.walk(diretorio):
        for nome_arquivo in files:
            if nome_arquivo.endswith(".txt"):
                caminho_arquivo = os.path.join(root, nome_arquivo)
                try:
                    with open(caminho_arquivo, "r", encoding="utf-8") as f:
                        linhas = f.readlines()
                        total_linhas += len([l for l in linhas if l.strip()])
                        total_arquivos += 1
                except Exception as e:
                    print(f"Erro ao ler {caminho_arquivo}: {e}")
    return total_linhas

# Salva frases de uma página, retorna o número de frases
def processar_pagina(titulo):
    wikipedia.set_lang("pt")
    try:
        conteudo = wikipedia.page(titulo).content
        conteudo = re.split(r"==+\s*(Ligações externas|Referências|Ver também|Bibliografia)\s*==+", conteudo)[0]
        # Remove subtítulos do tipo === Seção === ou == Seção ==
        conteudo = re.sub(r"^={2,}.*?={2,}$", "", conteudo, flags=re.MULTILINE)      
        # Tokeniza em frases
        frases = sent_tokenize(conteudo, language="portuguese")
        # Remove frases que só contêm espaços, tabs ou caracteres invisíveis
        frases = [re.sub(r"\s+", " ", f.strip()) for f in frases if f.strip() and not re.match(r"^={2,}.*={2,}$", f.strip())]
        if len(frases) < 3:
            return 0  # ignora textos muito curtos

        nome_arquivo = limpar_nome_arquivo(titulo)
        caminho = os.path.join(PASTA_DESTINO, nome_arquivo)

        with open(caminho, "w", encoding="utf-8") as f:
            for frase in frases:
                f.write(frase + "\n")
        return len(frases)
    except Exception as e:
        print(f"Erro ao processar '{titulo}': {e}")
        # pass
        # sys.exit(1)
        return 0
# Obtém títulos aleatórios relacionados
def expandir_titulos(temas_base, limite=5000):
    titulos = set()
    for tema in temas_base:
        try:
            relacionados = wikipedia.search(tema, results=500)
            titulos.update(relacionados)
        except Exception:
            continue
        if len(titulos) >= limite:
            break
    return list(titulos)

def save_list_to_txt(data_list, filename):
    with open(filename, 'w') as file:
        for item in data_list:
            file.write(str(item) + '\n')

# Geração e salvamento de frases
def coletar_frases_ate_limite(temas_base,meta_frases=1_000_000, n_jobs=-1):
    total_frases = contar_total_linhas_txt(PASTA_DESTINO)
    while total_frases < meta_frases:
        titulos_processados = []
        for _,_, temas in os.walk(PASTA_DESTINO):
            for temaa in temas:
                titulos_processados.append(temaa.replace('.txt',''))
        titulos_processados = [t.replace('_',' ') for t in titulos_processados]

        for tema in tqdm(temas_base):
            candidatos = sorted(expandir_titulos([tema], limite=1000))            
            candidatos = [t for t in candidatos if t not in titulos_processados]
            print(f"Processando {len(candidatos)} títulos para '{tema}'") # len()

            resultados = Parallel(n_jobs=n_jobs)(
                delayed(processar_pagina)(titulo) for titulo in candidatos
            )
            total_frases += sum(resultados)
            print(f"Total de frases coletadas: {total_frases}")
            if tema==temas_base[-1]:
                print(f"\n✅ Coleta finalizada com {total_frases} frases.")
                sys.exit(1)    

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [78]:
import os

def concatenate_txt_files(directory, output_file):
    # Abre o arquivo de saída em modo de escrita
    with open(output_file, 'w', encoding='utf-8') as outfile:
        # Itera sobre todos os arquivos no diretório
        for filename in os.listdir(directory):
            # Verifica se o arquivo tem a extensão .txt
            if filename.endswith('.txt'):
                file_path = os.path.join(directory, filename)
                # Abre o arquivo .txt em modo de leitura
                with open(file_path, 'r', encoding='utf-8') as infile:
                    # Lê o conteúdo do arquivo e escreve no arquivo de saída
                    outfile.write(infile.read())
                    # Adiciona uma linha em branco entre os arquivos
                    outfile.write('\n')

# Exemplo de uso
concatenate_txt_files(PASTA_DESTINO, 'arquivo_geral.txt')

In [93]:
def clean_file(input_file, output_file):
    # Abre o arquivo de entrada para leitura e o arquivo de saída para escrita
    with open(input_file, 'r', encoding='utf-8') as infile, open(output_file, 'w', encoding='utf-8') as outfile:
        # Itera sobre cada linha do arquivo de entrada
        for line in infile:
            # Remove espaços em branco no início e no final da linha
            stripped_line = line.strip()
            # Verifica se a linha não está vazia e não contém a palavra 'displaystyle'
            keywords = ['displaystyle', 'textstyle', 'sqrt', 'fract','=','Γ', 'Δ', 'Ε', 'Ζ', 'Η', 'Θ',
                        'Ι', 'Κ', 'Λ', 'Μ', 'Ν', 'Ξ', 'Ο', 'Π', 'Ρ',
                        'Σ', 'Τ', 'Υ', 'Φ', 'Χ', 'Ψ', 'Ω','alpha', 'gamma', 'epsilon', 'zeta', 'theta',
                        'iota', 'kappa', 'lambda', 'omicron','rho',
                        'sigma','upsilon', 'phi','φ']
            if all(keyword not in stripped_line for keyword in keywords) and len(stripped_line)>50:
                # Se nenhuma das strings estiver presente em stripped_line, execute o código aqui
                outfile.write(line)
                

# Exemplo de uso
clean_file('arquivo_geral.txt', 'arquivo_limpo.txt')

In [77]:
temas_base = [
    "Ciência", "História", "Tecnologia", "Educação", "Geografia",
    "Matemática", "Física", "Brasil", "Saúde", "Economia", "Literatura",
    "Arte", "Política", "Religião", "Música", "Cinema", "Esportes",
    "Meio ambiente", "Biologia", "Química", "Filosofia", "Sociologia",
    "Psicologia", "Astronomia", "Direito", "Engenharia", "Antropologia",
    "Computação", "Ecologia", "Medicina", "Linguística", "Arquitetura",
    "Mitologia", "Guerra", "Agricultura", "Climatologia", "Demografia",
    "Genética", "Robótica", "Educação física", "Paleontologia", "Geologia"
]
coletar_frases_ate_limite(temas_base)

  0%|          | 0/42 [00:00<?, ?it/s]

Processando 177 títulos para 'Ciência'


  2%|▏         | 1/42 [00:11<08:10, 11.98s/it]

Total de frases coletadas: 766015
Processando 181 títulos para 'História'


  5%|▍         | 2/42 [00:25<08:37, 12.93s/it]

...

 98%|█████████▊| 41/42 [10:04<00:14, 14.22s/it]

Total de frases coletadas: 1215028
Processando 257 títulos para 'Geologia'


 98%|█████████▊| 41/42 [10:20<00:15, 15.13s/it]

Total de frases coletadas: 1220899

✅ Coleta finalizada com 1220899 frases.


SystemExit: 1

Total de arquivos .txt: 995
Total de frases (linhas não vazias): 134118


134118

In [ ]:
titulos_processados = []
for _,_, temas in os.walk(r'C:\Users\Administrator\Desktop\Repositórios\SimCSE\textos_wikipedia'):
    for temaa in temas:
        titulos_processados.append(temaa.replace('.txt',''))